In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
current_dir = os.getcwd()
libs_path = os.path.abspath(os.path.join(current_dir, "..", "libs"))
if libs_path not in sys.path:
    sys.path.append(libs_path)

In [2]:
import pandas as pd
from IPython.display import display
from ipywidgets import interact
import numpy as np
pd.set_option('display.precision', 3)
import data_processing as dp

In [3]:
res=dp.load_datasets()

SelectMultiple(description='Датасеты:', layout=Layout(width='500px'), options={'.ipynb_checkpoints': 'E:\\stud…

Button(description='ОК', style=ButtonStyle())

Output()

In [4]:
datasets = []

for fn in res.options:

    if res.options[fn] not in res.value:
        continue
    dataset = dp.Dataset(pd.read_csv(res.options[fn]), name=fn)

    dataset.target_features = ["Cover_Type"]
    datasets.append(dataset)

In [5]:
for d in datasets:
    print(d.name)



df_enhanced.csv_scaled.csv


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Input, MaxPooling1D, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

def create_mlp_model(input_shape, num_classes, optimizer='adam', learning_rate=0.001, dense_units=[128, 64]):
    model = Sequential()
    model.add(Input(shape=(input_shape,)))
    for du in dense_units:
        model.add(Dense(du, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    if optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    else:
        opt = Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_1d_cnn_model(input_shape, num_classes, filters=[128, 256],
                         kernel_size=3, pool_size=2, use_batch_norm=True, dense_units=[128, 64],
                         optimizer='adam', learning_rate=0.001):
    model = Sequential()
    model.add(Input(shape=(input_shape, 1))) # input_shape уже должен быть (num_features, 1)
    i=0
    for f in filters:
        model.add(Conv1D(filters=f, kernel_size=kernel_size, activation='relu', padding='same'))
        if use_batch_norm:
            model.add(BatchNormalization())
        # Добавляем MaxPooling1D, если есть смысл и достаточное количество признаков
        if input_shape > pool_size * (i + 1) : # Убедимся, что размерность не станет слишком маленькой
            model.add(MaxPooling1D(pool_size=pool_size))
        else:
            # Если MaxPooling1D может сделать размерность 0, пропускаем его
            # или можно настроить логику по-другому, например, убрать padding='same'
            # и добавить его только на первом слое
            pass # Пропускаем MaxPooling, если он приведет к 0 размерности
        i+=1
    model.add(Flatten())
    for du in dense_units:
        model.add(Dense(du, activation='relu'))
        if use_batch_norm: # Можно добавить BatchNormalization и здесь, но это опционально
            model.add(BatchNormalization())

    model.add(Dense(num_classes, activation='softmax'))

    # Выбор оптимизатора
    if optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    else:
        opt = Adam(learning_rate=learning_rate) # Default

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


    

In [7]:
from itertools import product

def all_combinations(args_dict):
    # Получаем имена аргументов и их возможные значения
    arg_names = args_dict.keys()
    values = args_dict.values()
    res = []
    # Перебираем все комбинации значений
    for combination in product(*values):
        # Создаем словарь с аргументами
        kwargs = dict(zip(arg_names, combination))
        # Вызываем функцию с текущей комбинацией
        res.append(kwargs)
    return res



In [ ]:

from scikeras.wrappers import KerasClassifier
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)

for d in datasets:
    d.target_features
    temp_name = d.name
    num_classes = int(d[d.target_features].nunique()) # ИСПРАВЛЕНИЕ: Количество уникальных классов
    assert num_classes == 7
    temp = []
    for i in range(num_classes):
        temp.append(d.target_features[0] + f"_{i}")
    #d.target_features = 
    d[temp] = to_categorical(d[d.target_features].values - 1, num_classes=7) # Эта строка остается
    d = d.drop(d.target_features, axis=1)
    d = dp.Dataset(d)
    d.target_features = temp
    d.name = temp_name
    
    d.split_data(random_state=1337)

    all_models = []
    params = {"optimizer" : ['sgd', 'adam', 'rmsprop'], "learning_rate" : [0.00001, 0.0001], "dense_units":[[128, 64, 32], [128, 64], [128], [64]]}
    params_1d_cnn = {"kernel_size" : [3], "pool_size": [2], "use_batch_norm": [False, True], "filters":[[128, 256], [64, 128, 256]]}
    params_mlp = {}
    params_default = {"input_shape": [d.shape[1] - num_classes], "num_classes" : [num_classes]}
    
    
    mlp_comb = all_combinations({**params_default, **params, **params_mlp})
    cnn_comb = all_combinations({**params_default, **params, **params_1d_cnn})
    #print(len(mlp_comb))
    print(len(cnn_comb))

    iii = 0
    #for el in mlp_comb:
    for el in cnn_comb:

        print(iii, el)
        iii+=1
        #model = create_mlp_model(**el)
        model = create_1d_cnn_model(**el)

        history = model.fit(d.X_train.values, d.Y_train.values, epochs=20, batch_size=64, callbacks=[early_stopping])
        with open('results3.txt', 'a', encoding='utf-8') as f:
            f.write(f"\n\n{d.name}; {el};")

        

        Y_train_real = model.predict(d.X_train.values, verbose=0)
        Y_test_real = model.predict(d.X_test.values, verbose=0)
        Y_valid_real = model.predict(d.X_valid.values, verbose=0)
       # print(np.isnan(Y_train_real).any())
       # print(np.isnan(Y_test_real).any())
       # print(np.isnan(Y_valid_real).any())

        AAA = [[np.argmax(Y_train_real, axis=1), np.argmax(d.Y_train.values, axis=1), "train"], [np.argmax(Y_test_real, axis=1), np.argmax(d.Y_test.values, axis=1), "test"], [np.argmax(Y_valid_real, axis=1), np.argmax(d.Y_valid.values, axis=1), "valid"]]
        with open('results3.txt', 'a', encoding='utf-8') as f:
            for viborka in AAA:
                try:
                    #print(type(viborka[0]), type(viborka[1]))
                    #print(accuracy_score(viborka[0], viborka[1]))
                    f.write(f"{viborka[2]}; acc= {accuracy_score(viborka[0], viborka[1])};\n")
                except Exception as e:
                    print(e)
                    f.write(f"{viborka[2]}; acc= accuracy_score_error;\n")
                try:

                    f.write(f"{viborka[2]}; bal_acc= {balanced_accuracy_score(viborka[0], viborka[1])};\n")
                except:
                    f.write(f"{viborka[2]}; bal_acc= balanced_accuracy_score_error;\n")

                try:

                    f.write(f"{viborka[2]}; f1_macro= {f1_score(viborka[0], viborka[1], average='macro', zero_division=0)};\n")
                except:
                    f.write(f"{viborka[2]}; f1_macro= f1_score_error;\n")

                try:

                    f.write(f"{viborka[2]}; f1_weighted= {f1_score(viborka[0], viborka[1], average='weighted', zero_division=0)};\n")
                except:
                    f.write(f"{viborka[2]}; f1_weighted= f1_score_error;\n")

                try:
                    f.write(f"{viborka[2]}; classification_report= {classification_report(viborka[0], viborka[1], target_names=[f'Class {i+1}' for i in range(num_classes)], zero_division=0)};\n")
                except:
                    f.write(f"{viborka[2]}; classification_report= classification_report_error;\n")

    


C:\Users\nikol\AppData\Local\Temp\ipykernel_6888\1502637709.py:11: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  num_classes = int(d[d.target_features].nunique()) # ИСПРАВЛЕНИЕ: Количество уникальных классов


96
0 {'input_shape': 58, 'num_classes': 7, 'optimizer': 'sgd', 'learning_rate': 1e-05, 'dense_units': [128, 64, 32], 'kernel_size': 3, 'pool_size': 2, 'use_batch_norm': False, 'filters': [128, 256]}
Epoch 1/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 27s 5ms/step - accuracy: 0.0816 - loss: 1.9582
Epoch 2/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - accuracy: 0.4570 - loss: 1.9143
Epoch 3/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.5024 - loss: 1.8847
Epoch 4/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.5028 - loss: 1.8577
Epoch 5/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.5033 - loss: 1.8315
Epoch 6/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.5032 - loss: 1.8065
Epoch 7/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.5048 - loss: 1.7814
Epoch 8/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.5024 - loss: 1.7566
Epoch 9/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.5035 - loss: 1.7310


S:\anaconda\envs\study\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
S:\anaconda\envs\study\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
S:\anaconda\envs\study\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


1 {'input_shape': 58, 'num_classes': 7, 'optimizer': 'sgd', 'learning_rate': 1e-05, 'dense_units': [128, 64, 32], 'kernel_size': 3, 'pool_size': 2, 'use_batch_norm': False, 'filters': [64, 128, 256]}
Epoch 1/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.4356 - loss: 1.9297
Epoch 2/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.5024 - loss: 1.8985
Epoch 3/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.5056 - loss: 1.8652
Epoch 4/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.5023 - loss: 1.8335
Epoch 5/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.5039 - loss: 1.8015
Epoch 6/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.5042 - loss: 1.7696
Epoch 7/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.5027 - loss: 1.7373
Epoch 8/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.5034 - loss: 1.7043
Epoch 9/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 23s 4ms/step - accuracy: 0.5050 - loss: 1.6694

S:\anaconda\envs\study\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
S:\anaconda\envs\study\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
S:\anaconda\envs\study\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


2 {'input_shape': 58, 'num_classes': 7, 'optimizer': 'sgd', 'learning_rate': 1e-05, 'dense_units': [128, 64, 32], 'kernel_size': 3, 'pool_size': 2, 'use_batch_norm': True, 'filters': [128, 256]}
Epoch 1/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 56s 10ms/step - accuracy: 0.2848 - loss: 2.0307
Epoch 2/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 59s 11ms/step - accuracy: 0.5298 - loss: 1.5218
Epoch 3/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 50s 9ms/step - accuracy: 0.6003 - loss: 1.3807
Epoch 4/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 63s 11ms/step - accuracy: 0.6428 - loss: 1.2870
Epoch 5/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 51s 9ms/step - accuracy: 0.6710 - loss: 1.2196  
Epoch 6/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 48s 9ms/step - accuracy: 0.6875 - loss: 1.1676
Epoch 7/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 44s 8ms/step - accuracy: 0.7018 - loss: 1.1248  
Epoch 8/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 50s 9ms/step - accuracy: 0.7113 - loss: 1.0857 
Epoch 9/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 47s 9ms/step - accuracy: 0.7208 - loss: 1.0

S:\anaconda\envs\study\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
S:\anaconda\envs\study\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
S:\anaconda\envs\study\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


5 {'input_shape': 58, 'num_classes': 7, 'optimizer': 'sgd', 'learning_rate': 1e-05, 'dense_units': [128, 64], 'kernel_size': 3, 'pool_size': 2, 'use_batch_norm': False, 'filters': [64, 128, 256]}
Epoch 1/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 27s 5ms/step - accuracy: 0.4318 - loss: 1.9208
Epoch 2/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 24s 4ms/step - accuracy: 0.5036 - loss: 1.8831
Epoch 3/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 31s 6ms/step - accuracy: 0.5027 - loss: 1.8462
Epoch 4/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 31s 6ms/step - accuracy: 0.5024 - loss: 1.8092
Epoch 5/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 31s 6ms/step - accuracy: 0.5027 - loss: 1.7720
Epoch 6/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 31s 6ms/step - accuracy: 0.5019 - loss: 1.7341
Epoch 7/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 31s 6ms/step - accuracy: 0.5025 - loss: 1.6947
Epoch 8/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 31s 6ms/step - accuracy: 0.5041 - loss: 1.6532
Epoch 9/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 31s 6ms/step - accuracy: 0.5023 - loss: 1.6103
Epo

S:\anaconda\envs\study\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
S:\anaconda\envs\study\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
S:\anaconda\envs\study\Lib\site-packages\sklearn\metrics\_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


6 {'input_shape': 58, 'num_classes': 7, 'optimizer': 'sgd', 'learning_rate': 1e-05, 'dense_units': [128, 64], 'kernel_size': 3, 'pool_size': 2, 'use_batch_norm': True, 'filters': [128, 256]}
Epoch 1/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 49s 9ms/step - accuracy: 0.2823 - loss: 2.0148
Epoch 2/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 48s 9ms/step - accuracy: 0.5805 - loss: 1.4257
Epoch 3/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 48s 9ms/step - accuracy: 0.6461 - loss: 1.2838
Epoch 4/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 48s 9ms/step - accuracy: 0.6755 - loss: 1.1945
Epoch 5/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 48s 9ms/step - accuracy: 0.6897 - loss: 1.1343
Epoch 6/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 48s 9ms/step - accuracy: 0.7019 - loss: 1.0860
Epoch 7/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 48s 9ms/step - accuracy: 0.7092 - loss: 1.0486
Epoch 8/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 48s 9ms/step - accuracy: 0.7144 - loss: 1.0197
Epoch 9/20
5528/5528 ━━━━━━━━━━━━━━━━━━━━ 49s 9ms/step - accuracy: 0.7210 - loss: 0.9887
Epoch 10

In [14]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]
